In [2]:
import pandas as pd
import sqlite3
import json
from pathlib import Path
Path('sqlite_movie.db').touch()


In [3]:
dcon = sqlite3.connect("sqlite_movie copy.db")
# Load the data into a DataFrame
cursor = dcon.cursor()
movies_df = pd.read_sql_query("SELECT * from movies", dcon)
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [4]:
links_df = pd.read_sql_query("SELECT * from links", dcon)
links_df

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
62418,209157,6671244,499546.0
62419,209159,297986,63407.0
62420,209163,6755366,553036.0
62421,209169,249603,162892.0


In [5]:
cluster_df = pd.read_sql_query("SELECT * from cluster", dcon)
cluster_df

,Unnamed: 0,movieId,Cluster,title,genres
0,0,1,107,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,50,Jumanji (1995),Adventure|Children|Fantasy
2,2,3,41,Grumpier Old Men (1995),Comedy|Romance
3,3,4,41,Waiting to Exhale (1995),Comedy|Drama|Romance
4,4,5,136,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...
7195,7195,9010,82,Love Me If You Dare (Jeux d'enfants) (2003),Drama|Romance
7196,7196,9011,170,Portrait of Jennie (1948),Drama|Fantasy|Mystery|Romance
7197,7197,9013,88,Secret Honor (1984),Drama
7198,7198,9014,85,Since You Went Away (1944),Drama|War


In [6]:
ratingssummary_df = pd.read_sql_query("SELECT * from ratingssummary", dcon)
ratingssummary_df

,Unnamed: 0,movieId,avg_rating,ratingcounts
0,0,1,3.893708,57309
1,1,2,3.251527,24228
2,2,3,3.142028,11804
3,3,4,2.853547,2523
4,4,5,3.058434,11714
...,...,...,...,...
59042,59042,209157,1.500000,1
59043,59043,209159,3.000000,1
59044,59044,209163,4.500000,1
59045,59045,209169,3.000000,1


In [7]:
#close connection
dcon.close()

In [8]:
#merging movies and cluster dataframes
final_df=movies_df.merge(cluster_df, how='right', on='movieId')
final_df=final_df.drop(columns='title_y')
final_df=final_df.drop(columns='genres_y')
final_df=final_df.drop(columns='Unnamed: 0')
final_df.set_index('movieId', inplace=True)
final_df

,title_x,genres_x,Cluster
movieId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,107
2,Jumanji (1995),Adventure|Children|Fantasy,50
3,Grumpier Old Men (1995),Comedy|Romance,41
4,Waiting to Exhale (1995),Comedy|Drama|Romance,41
5,Father of the Bride Part II (1995),Comedy,136
...,...,...,...
9010,Love Me If You Dare (Jeux d'enfants) (2003),Drama|Romance,82
9011,Portrait of Jennie (1948),Drama|Fantasy|Mystery|Romance,170
9013,Secret Honor (1984),Drama,88


In [9]:
#merging final movie with ratings
final_df_2=final_df.merge(ratingssummary_df, how='right', on='movieId')
final_df_2=final_df_2.drop(columns='ratingcounts')
final_df_2=final_df_2.drop(columns='Unnamed: 0')
final_df_2.set_index('movieId', inplace=True)
final_df_2

,title_x,genres_x,Cluster,avg_rating
movieId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,107.0,3.893708
2,Jumanji (1995),Adventure|Children|Fantasy,50.0,3.251527
3,Grumpier Old Men (1995),Comedy|Romance,41.0,3.142028
4,Waiting to Exhale (1995),Comedy|Drama|Romance,41.0,2.853547
5,Father of the Bride Part II (1995),Comedy,136.0,3.058434
...,...,...,...,...
209157,NaN,NaN,NaN,1.500000
209159,NaN,NaN,NaN,3.000000
209163,NaN,NaN,NaN,4.500000


In [10]:
#last merge to get links
final_df_3=final_df_2.merge(links_df, how='right', on='movieId')
#final_df_3.reset_index(drop=True, inplace=True)
#final_df_3.set_index('movieId', inplace=True)
final_df_3.dropna(inplace=True)
final_df_3

,movieId,title_x,genres_x,Cluster,avg_rating,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,107.0,3.893708,114709,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,50.0,3.251527,113497,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,41.0,3.142028,113228,15602.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,41.0,2.853547,114885,31357.0
4,5,Father of the Bride Part II (1995),Comedy,136.0,3.058434,113041,11862.0
...,...,...,...,...,...,...,...
8295,9010,Love Me If You Dare (Jeux d'enfants) (2003),Drama|Romance,82.0,3.836921,364517,8424.0
8296,9011,Portrait of Jennie (1948),Drama|Fantasy|Mystery|Romance,170.0,3.674699,40705,32558.0
8298,9013,Secret Honor (1984),Drama,88.0,3.620000,88074,50034.0
8299,9014,Since You Went Away (1944),Drama|War,85.0,3.625000,37280,42401.0


In [37]:
#converting dataframe into json
js = final_df_3.to_json(orient = 'columns')
data=final_df_3.to_json("/Users/attiyawaqqas/Documents/GitHub/Movie_Application/js/data.json", orient = 'records')